# FVCA

In [ ]:
from trustutils import run

run.introduction("Yannick Gorsse")
run.description("")
run.TRUST_parameters()

In [ ]:
from trustutils import run
from string import Template
import os, sys
run.useMEDCoupling()
import medcoupling as mc

nb_meshes_max = 2 # mettre a 10 si on veut faire tous les maillages

run.reset()
m2d = {
    # 2D meshes
    "Cartesian":      (["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"], 7), 
    "Triangles":      (["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"], 6),
    "Locally_Refined":(["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"], 7),
    "Quadrangles":    (["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"], 7),
    "Kershaw2D":      (["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"], 6),
    "Polygons":       (["PolyMAC_P0P1NC", "PolyMAC_P0", "PolyMAC"], 6),
}
m3d = {
    # 3D meshes
    "Hexa":           (["PolyMAC", "PolyMAC_P0", "PolyMAC"], 5), # TODO : ajouter VDF, mais veut pas unpolyze...
    "Tetra":          (["PolyMAC", "PolyMAC_P0", "PolyMAC"], 7),
    "Prism":          (["PolyMAC", "PolyMAC_P0", "PolyMAC"], 4),
    "Kershaw":        (["PolyMAC", "PolyMAC_P0", "PolyMAC"], 4),
    "CheckerBoard":   (["PolyMAC", "PolyMAC_P0", "PolyMAC"], 5),
    "PrismHexa":      (["PolyMAC", "PolyMAC_P0", "PolyMAC"], 4),
    # "Random":         (["PolyMAC", "PolyMAC_P0"], 4),
    "LocRaf":         (["PolyMAC", "PolyMAC_P0", "PolyMAC"], 5),
    "Voronoi":        (["PolyMAC", "PolyMAC_P0", "PolyMAC"], 5),
}
mxd = { 2 : m2d, 3 : m3d }
tests = {
    "bercovier_engelman" : 2,
    "taylor_green_3d" : 3,
    "navier_stokes_2d" : 2,
    "navier_stokes_3d" : 3
}

meshes_dir = os.environ["TRUST_ROOT"] + "/Validation/Rapports_automatiques/Verification/PolyMAC/share/meshes"

# Trigger the copy of the src directory into the build directory:
run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)
origin, build_dir = os.getcwd(), run.BUILD_DIRECTORY
os.chdir(build_dir)
list_dis = []
for test, dim in tests.items():
    for m, (dis, ns) in mxd[dim].items():
        for d in dis:
            for n in range(1, min(ns, nb_meshes_max) + 1):
                os.system(f"mkdir -p {test}/{m}/{d}/{n}")
                with open(f"{test}.data", "r") as file: filedata = Template(file.read())
                result = filedata.substitute({"dis" : d})
                with open(f"{test}/{m}/{d}/{n}/jdd.data", "w") as file: file.write(result)  
                if not d.startswith("Poly"):
                    os.system(f"gunzip -c {meshes_dir}/{m}/jdd_{n}/mesh.med.gz >{test}/{m}/{d}/{n}/mesh.med")
                    meshname = f"{test}/{m}/{d}/{n}/mesh.med"
                    mfum = mc.MEDFileUMesh(meshname)
                    mfum.getMeshAtLevel(0).unPolyze()
                    mfum.write(f"{test}/{m}/{d}/{n}/mesh.med", 2)
                else: os.system(f"gunzip -c {meshes_dir}/{m}/jdd_{n}/mesh.med.gz >{test}/{m}/{d}/{n}/mesh.med")
                np = min(int(os.environ["TRUST_NB_PROCS"]), 8 if n < 2 else 128)
                os.system(f"cp post_run {test}/{m}/{d}/{n}/; cd {test}/{m}/{d}/{n}/; make_PAR.data jdd {np} >/dev/null 2>&1")
                #echo $mesh/jdd_${n}/cas.data >> ll
                run.addCase(f"{test}/{m}/{d}/{n}", "PAR_jdd.data", np)
                list_dis.append(f"{test}/{m}/{d}")

with open("extract_convergence", "r") as file: fileconv = Template(file.read())
result = fileconv.safe_substitute(list_dis=" ".join(sorted(list(set(list(list_dis))))))
with open("extract_convergence", "w") as file: file.write(result)
os.chdir(origin)

# run.printCases()
run.runCases()
run.executeScript("extract_convergence")

In [ ]:
from trustutils import plot
import matplotlib.colors as mcolors
import pandas as pd
from matplotlib.lines import Line2D
c = list(mcolors.TABLEAU_COLORS.keys())
linestyle = {"PolyMAC_P0P1NC": ("-", "o"), "PolyMAC_P0" : ("-.", "v"), "VEFPreP1B" : ("dashdot", "s"), "PolyMAC" : ("dashed", "d")}

lines_and_markers = {test : [Line2D([0], [0], color=c[k]) for k, _ in enumerate(mxd[dim].items())] for test, dim in tests.items()}
for test in tests.keys(): lines_and_markers[test] += [Line2D([0], [0], color="black", marker=m, linestyle=s) for _, (s, m) in linestyle.items()]
names_in_legend = {test : list(mxd[dim].keys()) for test, dim in tests.items()}
for test in tests.keys(): names_in_legend[test] += list(linestyle.keys())

def make_custom_legend(graph, test):
    try:
        for ax in graph.axs:
            ax.legend(lines_and_markers[test], names_in_legend[test], loc='upper center', ncol=4, bbox_to_anchor=(0.5, -0.1))
    except:
        graph.subplot.legend(lines_and_markers[test], names_in_legend[test], loc='upper center', ncol=4, bbox_to_anchor=(0.5, -0.1))


## Ordres de convergence

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

def func(x, a, b): return a * x ** b
for test, dim in tests.items():
    a = plot.Graph(f"{test}", nY=2)
    for k, (m, (dis, ns)) in enumerate(mxd[dim].items()):
        for d in dis:
            for i, var in enumerate(["v", "p"]):
                a.addPlot(i)
                x_ = plot.loadText(f"{test}/{m}/{d}/conv_glob_{var}")
                x, y = (np.power(x_[0], 1.0 / dim)).tolist(), x_[1].tolist()
                try: (_, b), _ = curve_fit(func, x, y)
                except: b = 0
                a.add(x, y, label=None, color=c[k], linestyle=linestyle[d][0], marker=linestyle[d][1], markersize=12)

                a.label(f"$N_i^{{1/{dim}}}$",f"erreur L2 - {var}")
                a.scale(xscale='log',yscale='log')
    make_custom_legend(a, test)


## Temps de calcul

In [ ]:
for test, dim in tests.items():
    a = plot.Graph(f"{test}", nY=2)
    for k, (m, (dis, _)) in enumerate(mxd[dim].items()):
        for d in dis:
            for i, var in enumerate(["v", "p"]):
                a.addPlot(i)
                x_ = plot.loadText(f"{test}/{m}/{d}/conv_glob_{var}")
                x, y = x_[2].tolist(), x_[1].tolist()
                a.add(x, y, label=None, color=c[k], linestyle=linestyle[d][0], marker=linestyle[d][1], markersize=12)

                a.label(f"Temps de calcul", f"erreur L2 - {var}")
                a.scale(xscale='log',yscale='log')
    make_custom_legend(a, test)
